![title](qgr.png)

# Principal Component Analysis Tutorial

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.__version__

'1.21.2'

In [3]:
pd.__version__

'1.3.2'

### Welcome to Quant Guild Research!

In this notebook we will be walking you through principal component analysis, prior knowledge in this domain is not required but definitely won't hurt!  

Let us first introduce the problem space...

Suppose you had some data:

In [4]:
# Measurable variables x and y
x = np.arange(0, 10, 1)
y = np.arange(0,5, .5)
z = (x+y)/2

# Aggregated data
data = pd.DataFrame(columns=['x', 'y', 'z'])
data['x'] = x
data['y'] = y
data['z'] = z
data.head()

,x,y,z
0,0,0.0,0.00
1,1,0.5,0.75
2,2,1.0,1.50
3,3,1.5,2.25
4,4,2.0,3.00


These variables may represent almost anything (stock price, lagged price, earnings, etc).  Nevertheless, the main idea we want to illustrate is that each variable is not orthogonal to another; that is, the information encoded into one variable may also be encoded into another.  

Think about a stock's price today and a stock's price yesterday, the price today has new information encoded into it, but also overlaps with some of the information from the previous day.  Utilizing this type of information in a model creates a combination of noise and redundancy that overwhelming hurts during the modeling process.

Covariance is a measure of one variables directional relationship with another - we can use a covariance matrix to get a quantitative measure of this idea:

$$cov(x, x) = var(x)$$

$$\Sigma = \begin{bmatrix}
var(x) & cov(x, y) & cov(x, z)\\
cov(y, x) & var(y) & cov(y, z)\\
cov(z, x) & cov(z, y) & var(z)
\end{bmatrix}$$

In Python we can retrieve the covariance matrix directly from a Pandas DataFrame:

In [5]:
data.cov()

,x,y,z
x,9.166667,4.583333,6.87500
y,4.583333,2.291667,3.43750
z,6.875000,3.437500,5.15625


We can see quite clearly that the directional relationship of measured variables x,y,z is far from indepenedent (given by non-zero covariances).

What if we could diagonalize the covariance matrix? ...

$$\Sigma_I = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}$$

If this was the case, it would imply that each variable contributes unique information - exactly what we are looking for to reduce noise and redundancy! 

So now we have defined a goal: find a linear transformation that allows us to arrive at a diagonalized covariance matrix.  The linear transformation applied to the data matrix in efforts to diagonalize the covariance matrix creates our principal components, hence principal component analysis.

To accomplish this we will use some tricks from linear algebra, after de-meaning we can compute the covariance matrix as:

$$\Sigma_X = \frac{X X^T}{n-1}$$

Where $X$ is our data matrix and $n$ is the number of samples:

In [6]:
# De-meaning the data
ddata = (data - data.mean())
ddata.head()

,x,y,z
0,-4.5,-2.25,-3.375
1,-3.5,-1.75,-2.625
2,-2.5,-1.25,-1.875
3,-1.5,-0.75,-1.125
4,-0.5,-0.25,-0.375


#### The Covariance Matrix

In [7]:
# Computing the covariance matrix directly
np.dot(ddata.values.T, ddata.values)/(ddata.shape[0]-1)

array([[9.16666667, 4.58333333, 6.875     ],
       [4.58333333, 2.29166667, 3.4375    ],
       [6.875     , 3.4375    , 5.15625   ]])

Since the covariance matrix symmetric we can compute the eigen decomposition or singular value decomposition:

$$X = USV^{T}$$
$$or$$
$$\Sigma_X = X X^T= Z \Lambda Z^{-1}$$

#### Eigen Decomposition

In [47]:
# Eigen decomposition for eigen vectors
l, Z = np.linalg.eig(np.dot(ddata.values.T, ddata.values)/(ddata.shape[0]-1))
l = np.diag(l)
Z

array([[ 0.74278135,  0.66953406, -0.0900247 ],
       [ 0.37139068, -0.41202096, -0.76909479],
       [ 0.55708601, -0.61803144,  0.6327628 ]])

In [48]:
# Reconstruction of covariance matrix via eigen decomposition
np.dot(np.dot(Z, l), Z.T)

array([[9.16666667, 4.58333333, 6.875     ],
       [4.58333333, 2.29166667, 3.4375    ],
       [6.875     , 3.4375    , 5.15625   ]])

#### Singular Value Decomposition

In [49]:
# Singular value decomposition for eigen vectors
U, S, VT = np.linalg.svd(np.dot(ddata.values.T, ddata.values)/(ddata.shape[0]-1))
S = np.diag(S)
U

array([[-0.74278135,  0.66951658,  0.00483894],
       [-0.37139068, -0.4059967 , -0.83500637],
       [-0.55708601, -0.6220243 ,  0.55021899]])

In [50]:
# Reconstruction of covariance matrix via singular value decomposition
np.dot(np.dot(U, S), VT)

array([[9.16666667, 4.58333333, 6.875     ],
       [4.58333333, 2.29166667, 3.4375    ],
       [6.875     , 3.4375    , 5.15625   ]])

Note that the sign is irrelevant and the primary difference is in the SVD which produces the square of eigenvalues and is sorted unlike the eigen decomposition.

To diagonalize the covariance matrix we will first consider eigen decomposition...

Now suppose we take our original data matrix $X$ and operate in this new basis $Z$, the data can be called $X_Z$

$$X_Z = Z^T X$$

Now consider the covariance under this new basis

$$\Sigma_{X_Z} = \frac{X_Z X_Z^T}{n-1}$$

Now let's substitute in for $X_Z$

$$\Sigma_{X_Z} = \frac{Z^T X (Z^T X)^T}{n-1} = \frac{Z^T X X^T Z}{n-1}$$

But we know that $X X^T$ is given in terms of the decompositions we arrived at above: $X X^T = Z \Lambda Z^{-1}$

$$\Sigma_{X_Z} = \frac{Z^TZ \Lambda Z^{-1} Z}{n-1}$$

Something really cool happens now (by construction) to $Z^T Z$ and $Z^{-1} Z$:

In [58]:
np.linalg.inv(Z.T)

array([[ 0.74278135,  0.66953406,  0.        ],
       [ 0.37139068, -0.52492263, -0.83967522],
       [ 0.55708601, -0.54276367,  0.55978348]])

In [60]:
Z

array([[ 0.74278135,  0.66953406, -0.0900247 ],
       [ 0.37139068, -0.41202096, -0.76909479],
       [ 0.55708601, -0.61803144,  0.6327628 ]])

In [62]:
np.round(np.dot(Z, np.linalg.inv(Z)), 1)

array([[ 1.,  0., -0.],
       [-0.,  1., -0.],
       [ 0., -0.,  1.]])

This means that $Z^T = Z^{-1}$ and $ZZ^T = I$ thus

$$\Sigma_{X_Z} = \frac{\Lambda}{n-1}$$

But what is $\Lambda$?  Diagonal!

In [14]:
np.round(l, 4)

array([[16.6146,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    , -0.    ]])

Naturally, we can acheive this same result with our singular value decomposition

In [15]:
np.round(S, 4)

array([[16.6146,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ]])

This draws an interesting connection between the eigen decomposition and singular value decomposition of the covariance matrix, mainly

$$\lambda_i = \sigma_i^2$$

The squared singular values are equivalent to the respective eigenvalue!

The derivation from singular value decomposition is even more straightforward:

$$X = U S V^T$$

$$X_V = V^T X^T$$

$$\Sigma_{X_V} = \frac{X_V X_V^T}{n-1} = \frac{V^T X^T (V^T X^T)^T}{n-1} = \frac{V^T (U S V^T)^T (V^T (U S V^T)^T)^T}{n-1} = \frac{V^T V S^T U^T U S V^T V}{n-1} = \frac{S^2}{n-1} = \frac{\Lambda}{n-1}$$

#### Let's get back to the root of the problem

In [16]:
data.head()

,x,y,z
0,0,0.0,0.00
1,1,0.5,0.75
2,2,1.0,1.50
3,3,1.5,2.25
4,4,2.0,3.00


We know that variables x and y contribute information to z (again, by construction) and we want to remove the overlapping information from the two variables

To accomplish this we transform the variables to operate in a new basis via eigen decomposition ($Z^T$) or singular value decomposition ($V^T$)

#### PCA via Eigen Decomposition of The Covariance Matrix

In [17]:
# Eigen way
l, Z = np.linalg.eig(ddata.cov())
zt = np.dot(Z.T, ddata.T)

Eigen Principal Components

In [18]:
zt_df = pd.DataFrame(zt.T)
zt_df.head()

,0,1,2
0,-6.058310,-5.412337e-16,-1.249001e-16
1,-4.712019,-3.469447e-16,1.249001e-16
2,-3.365728,-3.747003e-16,-6.938894e-17
3,-2.019437,-6.938894e-17,-4.163336e-17
4,-0.673146,-4.163336e-17,1.387779e-17


In [19]:
np.round(np.cov(zt), 4)

array([[16.6146,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ]])

#### PCA via Singular Value Decomposition of the Data Matrix

In [20]:
Ux, Sx, VTx = np.linalg.svd(ddata)

In [21]:
# SVD way
vt = np.dot(VTx, ddata.T)

SVD Principal Components

In [22]:
vt_df = pd.DataFrame(vt.T)
vt_df.head()

,0,1,2
0,-6.058310,1.054712e-15,6.245005e-17
1,-4.712019,9.436896e-16,-3.955170e-16
2,-3.365728,6.106227e-16,-7.632783e-17
3,-2.019437,3.885781e-16,-9.020562e-17
4,-0.673146,1.110223e-16,-2.081668e-17


These are our principal components!  Each variable in this new basis is orthogonal to one another and the information contributed is statistically independent!

In [23]:
np.round(np.cov(vt), 4)

array([[16.6146, -0.    ,  0.    ],
       [-0.    ,  0.    , -0.    ],
       [ 0.    , -0.    ,  0.    ]])

© 2022, Quant Guild